In [5]:
%pip install requests pandas python-dotenv

import os
from dotenv import load_dotenv
import requests
import pandas as pd
from io import StringIO


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Load .env file
load_dotenv()

# Get the token from the environment
token = os.getenv("PAT")

# Set up your request
url = "https://raw.githubusercontent.com/elijah-messmer/Wirehouse-Tracker/refs/heads/main/Commonwealth_Advisors.csv"
headers = {"Authorization": f"token {token}"}
r = requests.get(url, headers=headers)

# Load the CSV
df = pd.read_csv(StringIO(r.text))

# Reshape Data from Wide to Long
# ---------------------------------
# This stacks employment records into a format ideal for analysis.
df_long = pd.wide_to_long(
    df,
    stubnames=['firmName', 'registrationBeginDate', 'registrationEndDate', 'employmentStatus'],
    i='CRD',
    j='employment_record_id',
    sep='_',
    suffix='\\d+'
).reset_index()

# 3. Filter for the Target Firm
# -----------------------------
TARGET_FIRM = 'COMMONWEALTH FINANCIAL NETWORK'
cwn_df = df_long[df_long['firmName'] == TARGET_FIRM].copy()

# 4. Tally Starts and Leaves by Month (Using String Slicing)
# ----------------------------------------------------------
# Group by the first 7 characters of the date string (e.g., '2023-10').
monthly_starts = cwn_df.dropna(subset=['registrationBeginDate']).groupby(
    cwn_df['registrationBeginDate'].str[:7]
).size().rename('starts')

monthly_leaves = cwn_df.dropna(subset=['registrationEndDate']).groupby(
    cwn_df['registrationEndDate'].str[:7]
).size().rename('leaves')

# 5. Combine and Calculate Net Change
# -----------------------------------
net_flow = pd.concat([monthly_starts, monthly_leaves], axis=1).fillna(0).astype(int)
net_flow['net_change'] = net_flow['starts'] - net_flow['leaves']

# In your Python script (e.g., analyze_advisors.py)
net_flow.reset_index().to_json('results.json', orient='split', indent=4)

print(net_flow.sort_index())

         starts  leaves  net_change
1985-01       1       0           1
1986-04       1       0           1
1986-08       1       0           1
1986-11       1       0           1
1988-09       1       0           1
...         ...     ...         ...
2025-03      50      14          36
2025-04      61      17          44
2025-05      33      53         -20
2025-06      35      31           4
2025-07      33      29           4

[389 rows x 3 columns]
